In [ ]:
# df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()  -> for indexing multiindex

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer import OpEngineer, GroupOpEngineer
from realgam.quantlib.engineer.ta_engineer import TalibEngineer, GroupTalibEngineer
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'

In [2]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [3]:
stacked_hist = stacked_hist.reset_index().set_index(['ticker', 'date'])

In [4]:
tickers = stacked_hist.index.get_level_values('ticker').unique()[:4000]
small_hist = stacked_hist[stacked_hist.index.isin(tickers, level='ticker')]
A = stacked_hist[stacked_hist.index.isin(['A'], level='ticker')]

In [4]:
ae = AlphaEngineerV(stacked_hist)

In [7]:
s_time_chunk = time.time()
stacked_hist['alpha1'] = ae.alpha1(5, 20)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 302.45863914489746 sec


In [6]:
s_time_chunk = time.time()
stacked_hist['alpha1'] = ae.alpha1(5, 20)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 319.57942175865173 sec


In [5]:
s_time_chunk = time.time()
stacked_hist['alpha4'] = ae.alpha4(9)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 102.20251607894897 sec
Total time: 6.799350023269653 sec
Total time: 200.1494369506836 sec


In [5]:
s_time_chunk = time.time()
stacked_hist['alpha2'] = ae.alpha2()
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

/Users/punlee/miniconda3/envs/quant/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total time: 314.89851689338684 sec


In [19]:
volume = stacked_hist.volume.unstack('ticker')

In [6]:
from joblib import Parallel, delayed
from realgam.quantlib.engineer.utils import rank_last_value

In [43]:
def func(series):

    return series.rolling(10).apply(np.argmax, engine='cython', raw=True).add(1)

In [47]:
def rolling_rank(x, window, pct=False, min_prob=None):
    """
    Get the rolling rank of the last value according to the previous window of values.
    """
    # https://github.com/pandas-dev/pandas/issues/9481
    import bottleneck as bk
    norm_rank = bk.move_rank(x, window, axis=0) # [-1, 1]
    u = (norm_rank + 1) / 2 # [0, 1]
    if pct:
        if min_prob is None:
            min_prob = 1 / (window + 1)
            return u * (1 - 2 * min_prob) + min_prob # [min_prob, 1 - min_prob]
    rank = u * (window - 1)
    return np.round(rank)

In [52]:
s_time_chunk = time.time()
dummy = rolling_rank(volume, 10, pct=True)
pd.DataFrame(dummy, index=volume.index, columns=volume.columns)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 0.861375093460083 sec


In [44]:
s_time_chunk = time.time()
values = Parallel(n_jobs=-1)(delayed(func)(volume[col]) for col in volume)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 3.8957040309906006 sec


In [53]:
s_time_chunk = time.time()
volume.rolling(10).apply(rank_last_value, raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 121.04070401191711 sec


In [33]:
n_total_tickers = volume.shape[1]
batch_size = 20
s_time_chunk = time.time()
values = Parallel(n_jobs=-1)(delayed(func)(volume.iloc[:,i:i + batch_size ]) for i in range(0, n_total_tickers, batch_size))
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 5.859202861785889 sec


In [22]:
from numpy.lib.stride_tricks import sliding_window_view
sw = sliding_window_view(volume, 7, axis=0)
scores_np = (sw <= sw[..., -1:]).sum(axis=2) / sw.shape[-1]
scores_np_df = pd.DataFrame(scores_np, columns=volume.columns)

In [23]:
volume.shape

(2570, 10375)

In [25]:
scores_np

array([[0.14285714, 0.        , 0.        , ..., 0.        , 0.42857143,
        1.        ],
       [0.28571429, 0.        , 0.        , ..., 0.        , 0.71428571,
        1.        ],
       [0.85714286, 0.        , 0.        , ..., 0.        , 0.28571429,
        0.57142857],
       ...,
       [0.28571429, 0.28571429, 0.        , ..., 0.14285714, 0.28571429,
        0.        ],
       [0.14285714, 0.71428571, 0.        , ..., 0.14285714, 1.        ,
        0.        ],
       [1.        , 0.57142857, 0.        , ..., 1.        , 1.        ,
        0.        ]])

In [72]:
pd.concat(values, axis=1)

ticker,A,AA,AAAP,AAC,AAC.U,AACC,AACG,AACH,AACI,AACIU,...,GHRS,GHSEQ,GHSI,GHVIU,GI,GIA,GIA.U,GIAC,GIACU,GIB
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14,0.0,2.0,NaN,9.0,9.0,NaN,8.0,NaN,NaN,NaN,...,5.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2022-03-15,0.0,1.0,NaN,7.0,2.0,NaN,2.0,NaN,NaN,NaN,...,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2022-03-16,1.0,1.0,NaN,6.0,1.0,NaN,9.0,NaN,NaN,NaN,...,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [30]:
open = small_hist.openadj.unstack('ticker')
volume = small_hist.volume.unstack('ticker')
open.rolling(10).corr(volume)

ticker,A,AA,AAAP,AAC,AAC.U,AACC,AACG,AACH,AACI,AACIU,...,IRTC,IRWD,IS,ISAA,ISBC,ISCA,ISCIQ,ISDR,ISEE,ISHCQ
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14,0.363984,0.107623,NaN,0.320861,-0.212449,NaN,-0.392003,NaN,NaN,NaN,...,0.113561,-0.518841,0.578930,-0.027211,0.143323,NaN,NaN,0.754544,-0.238503,NaN
2022-03-15,0.485656,0.400572,NaN,0.360201,-0.161166,NaN,-0.355019,NaN,NaN,NaN,...,-0.054785,-0.320664,0.821113,0.010575,-0.120884,NaN,NaN,0.748907,-0.290549,NaN
2022-03-16,0.500228,0.463567,NaN,0.285395,-0.204080,NaN,0.204381,NaN,NaN,NaN,...,-0.286412,0.327993,0.909696,0.076618,0.076101,NaN,NaN,0.730427,-0.141502,NaN


In [18]:
small_hist = small_hist.reset_index().set_index(['ticker', 'date'])

In [9]:
goe = GroupOpEngineer(small_hist, 'ticker')
goe.ts_corr('open','volume',10)

NotImplementedError: isna is not defined for MultiIndex

In [20]:
small_hist.groupby('ticker')['volume'].rolling(10).corr(small_hist['openadj'])

ticker  ticker  date      
A       A       2012-01-03         NaN
                2012-01-04         NaN
                2012-01-05         NaN
                2012-01-06         NaN
                2012-01-09         NaN
                                ...   
ACER    ACER    2022-03-14    0.243021
                2022-03-15    0.388569
                2022-03-16   -0.425023
                2022-03-17   -0.297427
                2022-03-18   -0.259615
Length: 12243500, dtype: float64

In [ ]:
df = small_hist[['closeadj']].reset_index().set_index(['ticker','date'])

In [180]:
goe = GroupOpEngineer(df, 'ticker')

In [181]:
df['ts_ret_closeadj'] = goe.ts_ret('closeadj').values
df['ts_std20_closeadj'] = goe.ts_std('closeadj', 20).values

In [182]:
df.loc[df['ts_ret_closeadj'] < 0, 'closeadj'] = df['ts_std20_closeadj']

In [183]:
df.loc[df['ts_ret_closeadj'].isnull() |  df['ts_std20_closeadj'].isnull(), 'closeadj'] = np.nan

In [184]:
df['closeadj'] = np.power(df.closeadj, 2)

In [185]:
goe.set_df(df)

In [186]:
df['ts_argmax5_closeadj'] = goe.ts_argmax('closeadj', 5).values

In [189]:
goe.set_df(df)
df['cs_pctrank_ts_argmax5_closeadj'] = goe.cs_rank('ts_argmax5_closeadj', method='dense')

In [190]:
df

closeadj  ts_ret_closeadj  ts_std20_closeadj  \
ticker date                                                       
A      2012-01-03       NaN              NaN                NaN   
       2012-01-04       NaN        -0.007975                NaN   
       2012-01-05       NaN         0.022402                NaN   
       2012-01-06       NaN         0.010812                NaN   
       2012-01-09       NaN         0.026214                NaN   
...                     ...              ...                ...   
ACER   2022-03-14  0.000590        -0.024691           0.024288   
       2022-03-15  0.000581        -0.004219           0.024098   
       2022-03-16  6.502500         0.080508           0.030271   
       2022-03-17  6.604900         0.007843           0.030126   
       2022-03-18  7.075600         0.035019           0.030635   

                   ts_argmax5_closeadj  cs_pctrank_ts_argmax5_closeadj  
ticker date                                                             
A      2012-01-03                  NaN                             NaN  
       2012-01-04                  NaN                             NaN  
       2012-01-05                  NaN                             NaN  
       2012-01-06                  NaN                             NaN  
       2012-01-09                  NaN                             NaN  
...                                ...                             ...  
ACER   2022-03-14                  2.0                             3.0  
       2022-03-15                  1.0                             2.0  
       2022-03-16                  5.0                             6.0  
       2022-03-17                  5.0                             6.0  
       2022-03-18                  5.0                             6.0  

[122435 rows x 5 columns]

In [ ]:
print(data.loc[idx[['AAPL','GOOGL'],'2021-03-18'],])

In [168]:
df

closeadj  ts_ret_closeadj  ts_std20_closeadj  \
ticker date                                                       
A      2012-01-03       NaN              NaN                NaN   
       2012-01-04       NaN        -0.007975                NaN   
       2012-01-05       NaN         0.022402                NaN   
       2012-01-06       NaN         0.010812                NaN   
       2012-01-09       NaN         0.026214                NaN   
...                     ...              ...                ...   
ACER   2022-03-14  0.000590        -0.024691           0.024288   
       2022-03-15  0.000581        -0.004219           0.024098   
       2022-03-16  6.502500         0.080508           0.030271   
       2022-03-17  6.604900         0.007843           0.030126   
       2022-03-18  7.075600         0.035019           0.030635   

                   ts_argmax5_closeadj  cs_pctrank_ts_argmax5_closeadj  
ticker date                                                             
A      2012-01-03                  NaN                             NaN  
       2012-01-04                  NaN                             NaN  
       2012-01-05                  NaN                             NaN  
       2012-01-06                  NaN                             NaN  
       2012-01-09                  NaN                             NaN  
...                                ...                             ...  
ACER   2022-03-14                  2.0                            24.5  
       2022-03-15                  1.0                            16.0  
       2022-03-16                  5.0                            43.0  
       2022-03-17                  5.0                            42.5  
       2022-03-18                  5.0                            43.5  

[122435 rows x 5 columns]

In [ ]:
df.loc[idx[slice(None), '2012-02-07'],]['cs_pctrank_ts_argmax5_closeadj']

In [165]:
idx = pd.IndexSlice
df.loc[idx[slice(None), '2012-02-07'],]['cs_pctrank_ts_argmax5_closeadj'].unique()

array([0.44897959, 0.66326531, 0.10204082, 0.85714286, 0.23469388])

In [166]:
df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()

array([0.10204082, 0.87755102, 0.42857143, 0.67346939, 0.2244898 ])

In [27]:
c = df.closeadj.values
r = df['ts_ret_closeadj'].values
c[r < 0 ] = df['ts_std20_closeadj'].values

ValueError: NumPy boolean array indexing assignment cannot assign 122318 input values to the 55284 output values where the mask is true

In [29]:
c[r < 0]

array([23.882, 25.684, 25.776, ...,  2.43 ,  2.37 ,  2.36 ])